for unstructured data use ubuntu

In [1]:
import os
from getpass import getpass

from datasets import (
    load_dataset,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

d:\a27_YEARS_OLD\multimodal_rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

import os
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings , ChatGoogleGenerativeAI

In [ ]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:

from langchain_community.document_loaders import DirectoryLoader

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY')
ASTRA_DB_API_ENDPOINT = os.getenv('ASTRA_DB_API_ENDPOINT')
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRADB_API_TOKEN')
ASTRA_DB_KEYSPACE="default_keyspace"

In [ ]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [ ]:
root_dir="/theory/"

In [ ]:
loader = DirectoryLoader('/theory/')

In [ ]:

pdf_folder_path = f'{root_dir}/docs/'

In [ ]:
os.listdir(pdf_folder_path)

In [ ]:
# location of the pdf file/files.
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [ ]:

loaders

In [ ]:

index = VectorstoreIndexCreator().from_loaders(loaders)

In [ ]:

index.query('What is the tokenization in RAG?')

In [ ]:
docs = loader.load_and_split(text_splitter=splitter)

In [ ]:

index.query_with_sources('What is the tokenization in RAG?')

In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain.indexes import VectorstoreIndexCreator

In [ ]:


vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="astra_vector_demo",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)
   

In [ ]:
root_dir="/theory/"
pdf_folder_path = f'{root_dir}/docs/'
pdfs=os.listdir(pdf_folder_path)

In [ ]:
pdfs

In [ ]:
data=PyPDFLoader("/theory/docs/cag.pdf")

In [ ]:
data

In [ ]:

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [ ]:

data.load_and_split(text_splitter=splitter)

In [ ]:

docs=[]
for pdf in pdfs:
  data=PyPDFLoader(f"/content/data/{pdf}")
  docs.append(data)

In [ ]:
docs_from_pdf = docs.load_and_split(text_splitter=splitter)

In [ ]:

print(f"Documents from PDF: {len(docs_from_pdf)}.")
inserted_ids_from_pdf = vstore.add_documents(docs_from_pdf)
print(f"Inserted {len(inserted_ids_from_pdf)} documents.")

In [ ]:

retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [ ]:

prompt_template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be concise and to the point, and refrain from answering about other topics than philosophy.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

In [ ]:

philo_prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", max_tokens=1024)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | philo_prompt
    | llm
    | StrOutputParser()
)

In [ ]:

chain.invoke("How does Russel elaborate on Peirce's idea of the security blanket?")
     

In [ ]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:


from langchain_community.document_loaders import DirectoryLoader

In [ ]:
loader = DirectoryLoader('/theory/docs')

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [ ]:
docs = loader.load_and_split(text_splitter=splitter)

In [ ]:

len(docs)

In [ ]:

vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="multidoc_vector",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [ ]:

inserted_ids = vstore.add_documents(docs)

In [ ]:
print(f"\nInserted {len(inserted_ids)} documents.")

In [ ]:
prompt_template = """
You are an AI philosopher drawing insights from the roadmap of "rag," "llama3," and "genai."
Craft thoughtful answers based on this roadmap, mixing and matching existing paths.
Your responses should be concise and strictly related to the provided context.

ROADMAP CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

In [ ]:

prompt_template = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [ ]:

retriever

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", max_tokens=1024)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:

chain.invoke("can you tell me difference between rag and cag ?")